# Working with Data

In this notebook, we are going to work on the datasets that are going to be used in order to fine tune our NLP models.  
The different steps that we are going to follow are:

- Scraping ads data from Moroccan youtube products channels

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
def scrape_ads(url):
    # Launch the Chrome driver
    driver = webdriver.Chrome('C:\chromedriver\chromedriver')
    driver.get(url)

    # Scroll down to load more videos
    height = driver.execute_script("return document.documentElement.scrollHeight")
    start_time = time.time()
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(1)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == height:
            break
        height = new_height
        if time.time() - start_time >= 5:
            break

    # Scrape links to individual videos
    src = driver.page_source 
    soup = BeautifulSoup(src, 'lxml')
    pub = soup.find_all('a', {'id': 'video-title-link'})
    links=[]
    for link in pub:
        href = link.get('href')
        if href is not None:
            links.append("https://www.youtube.com/" + href)

    # Scrape ads from each individual video
    ads=[]
    for link in links[0:len(links)-1]:
        driver.get(link)
        time.sleep(2)
        src = driver.page_source 
        soup = BeautifulSoup(src, 'lxml')
        ad = soup.find('span', {'class' : 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap'})
        if ad:
            ads.append(ad.text)

    # Quit the driver and return the list of ads
    driver.quit()
    return ads


In [3]:
ads0 = scrape_ads("https://www.youtube.com/@orangemaroc/videos")

C:\Users\ADS\AppData\Local\Temp\ipykernel_15248\4002602606.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver\chromedriver')


In [5]:
ads1 = scrape_ads("https://www.youtube.com/@maroctelecom/videos")

C:\Users\ADS\AppData\Local\Temp\ipykernel_15248\4002602606.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver\chromedriver')


In [13]:
ads2 = scrape_ads("https://www.youtube.com/@BimoOfficiel/videos")

C:\Users\ADS\AppData\Local\Temp\ipykernel_15248\4002602606.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver\chromedriver')


In [11]:
ads3 = scrape_ads("https://www.youtube.com/@inwi/videos")

C:\Users\ADS\AppData\Local\Temp\ipykernel_15248\4002602606.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver\chromedriver')


In [38]:
df = pd.DataFrame({"ads": ads0 + ads1 + ads2 + ads3})

In [41]:
df

,ads
0,حصرياً و غير عند اورنج، عيش لفيبر فدارك ونتا ه...
1,تعرفوا معنا على سر الحرفة مع فاطمة، عمران و هش...
2,بفضل الخطوات ديالكم 🤗 وزعنا الاأنترنت على عدة ...
3,الساعة جديدة هادي ولا القديمة ؟ 😅 \nشكون فعائل...
4,حيت عندنا ديما نتا لول، تبرع بالماكس ديال السخ...
...,...
485,"inwi, en partenariat avec 2M, lance la 2ème sa..."
486,#inwi_m3ak avec les offres wifi f'dar généreus...
487,ila ghayate 31/12
488,Découvrez notre premier candidat pour le Prime...


In [40]:
df.to_csv("ads_before_cleaning.csv", index=False)

# Data cleaning

In [14]:
import re

In [26]:
def clean_text(text):
    # Remove emojis
    text = re.sub('[^\u0600-\u06FF\s]+', '', text)
    # Remove non-Arabic letters and numbers
    text = re.sub('[^؀-ۿ\s]+', '', text)
    # Remove extra spaces
    text = re.sub('\s+', ' ', text)
    return text.strip()

In [27]:
# apply the function to the ads column
df['ads_clean'] = df['ads'].apply(clean_text)

In [31]:
df.drop('ads', axis=1, inplace=True)

In [33]:
df.dropna(subset=['ads_clean'], inplace=True)

In [34]:
df

,ads_clean
0,حصرياً و غير عند اورنج، عيش لفيبر فدارك ونتا ه...
1,تعرفوا معنا على سر الحرفة مع فاطمة، عمران و هش...
2,بفضل الخطوات ديالكم وزعنا الاأنترنت على عدة جم...
3,الساعة جديدة هادي ولا القديمة ؟ شكون فعائلتك و...
4,حيت عندنا ديما نتا لول، تبرع بالماكس ديال السخ...
...,...
485,
486,
487,
488,


In [35]:
df = df[df['ads_clean'].apply(lambda x: not x.strip() == '')]

In [37]:
df.to_csv("ads.csv", index=False)